In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

# Fonction de Rosenbrock

On considère la fonction de Rosenbrock, qui est définie sur $\mathbb{R}^2$: $f(x_1,x_2)=(x_1-1)^2+100(x_1^2-x_2)^2$, et dont le minimum est égal à $0$ et est atteint en $(1,1)$. Les fonctions suivantes implémentent la fonction de Rosenbrock, son gradient, et sa hessienne.

In [ ]:
def rosenbrock(x):
    y = np.asarray(x)
    return np.sum((y[0] - 1)**2 + 100*(y[1] - y[0]**2)**2)

def rosenbrock_grad(x):
    y = np.asarray(x)
    grad = np.zeros_like(y)
    grad[0] = 400*y[0]*(y[0]**2-y[1]) + 2*(y[0]-1)
    grad[1] = 200*(y[1]-y[0]**2)
    return grad

def rosenbrock_hessian(x):
    y = np.asarray(x)
    return np.array(((1 - 4*100*y[1] + 12*100*y[0]**2, -4*y[0]*100), (-4*100*y[0],2*100)))

La fonction suivante trace les lignes de niveau de la fonction de Rosenbrock, ainsi que les éventuelles itérées fournies sous la forme d'une liste.

In [ ]:
def rosenbrock_plot(iterates=[]):
    nx = 1000 # number of discretization points along the x-axis
    ny = 1000 # number of discretization points along the y-axis
    a=-1.5; b=1.5 # extreme points in the x-axis
    c=-0.5; d=1.5 # extreme points in the y-axis
    X,Y = np.meshgrid(np.linspace(a,b,nx), np.linspace(c,d,ny))
    Z = (1-X)**2+100*(X**2-Y)**2
    plt.contour(X,Y,Z,np.logspace(-0.5,3.5,20,base=10), colors='gray')
    plt.scatter([1],[1],c='red')
    plt.xlim([a,b])
    plt.ylim([c,d])
    if iterates != []:
        plt.plot([x[0] for x in iterates], [x[1] for x in iterates])
    plt.show()

# Descente de gradient

**Question:** Implémenter la descente de gradient $x^{(t+1)}=x^{(t)}-\gamma\nabla f(x^{(t)})$, en incluant deux critères d'arrêt: un seuil $\varepsilon$ pour la norme du gradient $\left\|  \nabla f(x^{(t)}) \right\|_2$ en dessous duquel l'algorithme prend fin, et un nombre maximal d'itérations. Cela prendra la forme d'une fonction `gradient_descent` qui prend un argument: une fonction objectif $f$, un fonction gradient $\nabla f$, un pas $\gamma$, un seuil $\varepsilon$, un nombre maximal d'itérations (avec une valeur par défaut à 10000, par exemple); et qui renvoie une liste contenant les itérées successives. Par ailleurs, la fonction affichera après la dernière itération le temps écoulé (en millisecondes), le nombre d'itérations effectuées et la valeur de la fonction en la dernière itérée.

*On pourra utiliser `.append()` pour ajouter un élément à une liste, `np.linalg.norm()` pour calculer la norme euclidienne, ainsi que `datetime.now().timestamp()` qui renvoie l'heure actuelle en millisecondes.*

**Question:** En utilisant la fonction créée à la question précédente ainsi que `rosenbrock_plot()`, tracer graphiquement les itérées de la descente de gradient avec $(-1,1)$ pour point initial, différentes valeurs de $\gamma$, et avec un seuil $\varepsilon=10^{-3}$ pour la norme du gradient.  Commenter les résultats.

*L'array NumPy correspondant au vecteur $(-1,1)$ se définit par `np.array([-1,1])`.*

**Question:** Implémenter la descente de gradient normalisée $x^{(t+1)}=x^{(t)}-\gamma\frac{\nabla f(x^{(t)})}{\left\| \nabla f(x^{(t)}) \right\|}$, et tracer les 200 premières itérées avec $\gamma=5\times 10^{-2}$. Commenter. Cette méthode a-t-elle une chance de converger ?

# Règles d'Armijo et de Wolfe

Pour les algorithmes qui passent de l'itérée $x^{(t)}$ à $x^{(t+1)}$ en choisissant une direction $d_t$ et un pas $\gamma_t$ de sorte que $x^{(t+1)}=x^{(t)}+\gamma_td_t$ ($d_t=-\nabla f(x^{(t)})$ dans le cas de la descente de gradient), la règle d'Armijo préconise de trouver un pas $\gamma_t$ vérifiant:
$$f(x^{(t)}+\gamma_td_t)\leqslant f(x^{(t)})+c\gamma_t\left< \nabla f(x^{(t)}), d_t \right>,$$
où $0<c<1/2$ est un paramètre choisi à l'avance (en pratique, on peut choisir $c=10^{-4}$).
Interpréter cette condition, et expliquer pourquoi il existe toujours un pas $\gamma_t$ la vérifiant lorsque $f$ est de classe $\mathcal{C}^1$.

**Question:** Implémenter une fonction `armijo` prenant en argument (entre autres): $f$, $x^{(t)}$, $f(x^{(t)})$, $\nabla f(x^{(t)})$, $d_t$ et renvoyant $x^{(t)}+\gamma_t d_t$ où $\gamma_t$ vérifie la règle d'Armijo. La fonction pourra essayer différentes valeurs pour $\gamma_t$ en commençant par $\left< \nabla f(x^{(t)}), d_t \right> /(L\left\| d_t \right\|_2^2)$, où $L$ est un paramètre choisi à l'avance.

*On pourra utiliser la fonction `np.dot()` pour calculer le produit scalaire entre deux vecteurs*

Il existe une sophistication de la règle d'Armijo, appelée règle de Wolfe, qui requiert la condition *supplémentaire* suivante:
$$\left< d_t, \nabla f(x^{(t)}+\gamma_td_t) \right>\geqslant c_2\left< d_t, \nabla f(x^{(t)}) \right> ,$$
où $1/2 < c_2 < 1$ est un paramètre choisi à l'avance (en pratique, on peut choisir $c_2=0.99$). Interpréter cette condition et expliquer pourquoi il existe toujours un pas $\gamma_t$ vérifiant *les deux conditions*.

**Question:** Implémenter une fonction `wolfe` (similaire à `armijo`) renvoyant $x^{(t)}+\gamma_t d_t$ où $\gamma_t$ vérifie la règle de Wolfe.

**Question:** Modifier la fonction `gradient_descent` afin qu'elle offre la possibilité, à l'aide d'un argument spécifié par l'utilisateur, d'utiliser les règles d'Armijo ou de Wolfe. Utiliser les règles d'Armijo et de Wolfe pour la fonction de Rosenbrock avec toujours pour point initial $(-1,1)$ et un seuil $\varepsilon=10^{-3}$ pour la norme du gradient. Commenter les résultats.

# Méthode de Newton

**Question:** Rédiger une fonction `newton` implémentant la méthode Newton $x^{(t+1)}=x^{(t)}-\gamma_t(\nabla^2f(x^{(t)}))^{-1}\nabla f(x^{(t)})$ où le pas $\gamma_t$ peut être éventuellement déterminée à l'aide des règles d'Armijo ou de Wolfe. Commenter les résultats obtenus pour la fonction de Rosenbrock avec toujours pour point initial $(-1,1)$ et un seuil $\varepsilon=10^{-3}$ pour la norme du gradient.

*On pourra utiliser l'opérateur `@` qui correspond à la multiplication matricielle*

# Une régression linéaire aux moindres carrés

On charge un jeu de données réel de dimension 280 pour effectuer une régression linéaire aux moindres carrés.

In [3]:
data = pd.read_csv('https://joon-kwon.github.io/optim-mathsv/dataset.csv', header=None).values
A = data[:,:-1]
b = data[:,-1]

On définit la fonction objectif ainsi que son gradient.

In [4]:
def loss(x):
    return np.sum((A@x - b)**(2))

def loss_grad(x):
    return 2*A.T @ (A @ x - b)

def loss_hess(x):
    return A.T @ A

**Question:** Comparer les résultats obtenus par la descente de gradient (avec différentes options). On limitera le nombre d'itérations à 1000 (voire moins) afin de limiter le temps de calcul.

On essaye à présent le module `scipy.optimize` qui implémente de nombreux algorithmes d'optimisation.

In [ ]:
result = minimize(loss, np.zeros(A.shape[1]), method='Nelder-Mead', jac=loss_grad, hess=loss_hess)

**Question:** En changeant l'argument `method`, essayer les différents algorithmes dont la liste se trouve dans la documentation du module. Commenter les résultats.